In [3]:
import pandas as pd
import numpy as np

RUTA_BASE = '/home/luciano/orga-datos/v3/'

In [41]:
set_kaggle = pd.read_csv(RUTA_BASE + 'test_final_100k.csv')

In [38]:
vistas = pd.read_csv(RUTA_BASE + 'fiuba_3_vistas.csv', usecols=['idpostulante', 'idAviso'])
postulaciones = pd.read_csv(RUTA_BASE + 'fiuba_4_postulaciones.csv', usecols=['idpostulante', 'idaviso'])

In [5]:
set_kaggle.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [6]:
postulantes = pd.read_csv(RUTA_BASE + 'postulantes.csv')

In [47]:
postulantes = pd.merge(
    postulantes.set_index('idpostulante'), 
    vistas.groupby('idpostulante').agg('count').rename(columns={'idAviso':'cantidad_vistas'}), 
    left_index=True, right_index=True, 
    how='left'
)
postulantes.head()

,nivel_educativo,nivel_educativo_real,sexo,edad,edad_real,cantidad_vistas
idpostulante,,,,,,
0z5Dmrd,4.0,True,1.0,53.0,True,5.0
0z5JW1r,3.0,True,1.0,47.0,True,17.0
0z5VvGv,2.0,True,1.0,31.0,True,7.0
0zB01pE,3.0,True,1.0,29.0,True,5.0
0zB026d,2.5,True,-1.0,40.0,True,10.0


In [48]:
postulantes = pd.merge(
    postulantes, 
    postulaciones.groupby('idpostulante').agg('count').rename(columns={'idaviso':'cantidad_postulaciones'}), 
    left_index=True, right_index=True, 
    how='left'
)
postulantes = postulantes.reset_index()
postulantes.head()

,idpostulante,nivel_educativo,nivel_educativo_real,sexo,edad,edad_real,cantidad_vistas,cantidad_postulaciones
0,0z5Dmrd,4.0,True,1.0,53.0,True,5.0,2.0
1,0z5JW1r,3.0,True,1.0,47.0,True,17.0,7.0
2,0z5VvGv,2.0,True,1.0,31.0,True,7.0,48.0
3,0zB01pE,3.0,True,1.0,29.0,True,5.0,2.0
4,0zB026d,2.5,True,-1.0,40.0,True,10.0,10.0


In [49]:
postulantes_kaggle = pd.merge(postulantes, set_kaggle, on='idpostulante', how='right')

In [50]:
postulantes_kaggle = postulantes_kaggle.drop(axis=1, labels=['id', 'idaviso'])

In [51]:
postulantes_kaggle = postulantes_kaggle.drop_duplicates()
postulantes_kaggle.head()

,idpostulante,nivel_educativo,nivel_educativo_real,sexo,edad,edad_real,cantidad_vistas,cantidad_postulaciones
0,0zB0LYv,2.0,True,1.0,30.0,True,16.0,6.0
1,0zB0v4v,4.0,True,1.0,32.0,True,14.0,5.0
2,0zB1369,2.0,True,1.0,31.0,True,6.0,5.0
4,0zB35QY,2.0,True,1.0,30.0,True,70.0,9.0
6,0zB4Lbd,2.0,True,1.0,31.0,True,257.0,311.0


In [63]:
cuantil = lambda clave, min_c, max_c: (postulantes[clave] > min_c) & (postulantes[clave] <= max_c)
sample_postulantes = \
    postulantes[cuantil('cantidad_postulaciones', 0, 3)].sample(n=16161).append(\
    postulantes[cuantil('cantidad_postulaciones', 3, 10)].sample(n=16162)).append(\
    postulantes[cuantil('cantidad_postulaciones', 10, 27)].sample(n=16161)).append(\
    postulantes[cuantil('cantidad_postulaciones', 27, 3352)].sample(n=16162))

In [64]:
sample_postulantes['nivel_educativo_real'].value_counts()

True     60513
False     4133
Name: nivel_educativo_real, dtype: int64

In [65]:
postulantes_kaggle['nivel_educativo_real'].value_counts()

True     59929
False     4717
Name: nivel_educativo_real, dtype: int64

In [66]:
sample_postulantes['edad_real'].value_counts()

True     62915
False     1731
Name: edad_real, dtype: int64

In [67]:
postulantes_kaggle['edad_real'].value_counts()

True     62206
False     2440
Name: edad_real, dtype: int64

In [68]:
postulantes_kaggle['sexo'].value_counts()

 1.0    31286
-1.0    31037
 0.0     2323
Name: sexo, dtype: int64

In [69]:
sample_postulantes['sexo'].value_counts()

-1.0    33244
 1.0    29718
 0.0     1684
Name: sexo, dtype: int64

In [70]:
postulantes_kaggle['cantidad_postulaciones'].describe()

count    46649.000000
mean        27.307059
std         61.905937
min          1.000000
25%          3.000000
50%         10.000000
75%         27.000000
max       3352.000000
Name: cantidad_postulaciones, dtype: float64

In [71]:
sample_postulantes['cantidad_postulaciones'].describe()

count    64646.000000
mean        23.951861
std         47.913138
min          1.000000
25%          4.000000
50%         10.500000
75%         27.750000
max       3352.000000
Name: cantidad_postulaciones, dtype: float64

In [72]:
postulantes_kaggle['cantidad_vistas'].describe()

count    53097.000000
mean        43.416992
std         83.666729
min          1.000000
25%          6.000000
50%         17.000000
75%         46.000000
max       3955.000000
Name: cantidad_vistas, dtype: float64

In [73]:
sample_postulantes['cantidad_vistas'].describe()

count    50613.000000
mean        41.570071
std         73.922813
min          1.000000
25%          6.000000
50%         18.000000
75%         47.000000
max       3955.000000
Name: cantidad_vistas, dtype: float64

In [74]:
pd.DataFrame(sample_postulantes['idpostulante']).to_csv(RUTA_BASE + 'sample_postulantes.csv', index=False)